In [5]:
import numpy as np
#from sklearn.linear_model import LogisticRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD, Adam

# RC2 - Pregunta 5

In [6]:
TPr=0.362
FPr=0.016
TotNO=35948
TotYES=4052
TP = TotYES * TPr
FP = TotNO * FPr
TN = TotNO - FP
print(TP)
print(FP)
accuracy = (TN + TP)/(TotYES+TotNO)
print(accuracy)

1466.8239999999998
575.168
0.9209914


# RC2 - Pregunta 8

Calcular la probabillidad estimada por la regresión logística

In [7]:
import numpy as np
h = 0.5 * -3.706423 + 1 * -3.6671724 + 2.9557335

In [8]:
print(h)

-2.5646504000000006


In [9]:
y = 1 / (1 + np.exp(-h))

In [10]:
print(y)

0.07144840278053262


# RC2 - Pregunta 9

Calcular la loss (entropía cruzada)



In [15]:
#instancio una regresión lineal en Keras
model = Sequential()
model.add(Dense(1, input_shape=(2,), activation='sigmoid'))
model.compile(Adam(lr=0.01), loss='binary_crossentropy', metrics=['accuracy'])

l=[]
x=np.array([[-3.706423],[-3.6671724]]) #weights
y=np.array([2.9557335]) #array of biases
l.append(x)
l.append(y)
model.set_weights(l)
print(model.get_weights,'--',model.weights)
observ = np.array([[0.5, 1]])
print(observ)
print(model.predict_proba(observ))
proba_ok=model.predict_proba(observ)

<bound method Model.get_weights of <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000020C9186C080>> -- [<tf.Variable 'dense_2/kernel:0' shape=(2, 1) dtype=float32, numpy=
array([[-3.706423 ],
       [-3.6671724]], dtype=float32)>, <tf.Variable 'dense_2/bias:0' shape=(1,) dtype=float32, numpy=array([2.9557335], dtype=float32)>]
[[0.5 1. ]]
[[0.07144841]]


In [18]:
print(f'cross_entropy mal  clasificado: {-1*np.log(proba_ok) - (1-1)*np.log(1-proba_ok)}')
print(f'cross_entropy bien clasificado: {-0*np.log(proba_ok) - (1-0)*np.log(1-proba_ok)}')

cross_entropy mal  clasificado: [[2.6387796]]
cross_entropy bien clasificado: [[0.07412931]]


# RC2 - Pregunta 10

Se entrenó la siguiente regresión multinomial (softmax) y se obtuvieron los parámetros abajo indicados

matriz de coeficientes devuelta por keras:

coef_matrix

[

[ 1.2032 , 0.34239826, -0.3272417 ],

[ 1.4500312 , 0.54530275, -0.64213556]

]

bias

[-1.7683319, 1.8029612, -1.7188473]

In [20]:
#Creo un modelo de clasificación multiclase con función de activación softmax
input_features=2
output_size=3
model = Sequential()
model.add(Dense(units=output_size,input_dim=input_features, activation='softmax', name="Capa_Oculta"))
model.summary()
lr = 0.04 

# Lo compilo, notar que en vez de binary_crossentropy va categorical_crossentropy
model.compile(loss = 'categorical_crossentropy', optimizer=Adam(lr=lr), metrics=['accuracy'])

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Capa_Oculta (Dense)          (None, 3)                 9         
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________


In [21]:
#Seteo los pesos y bias del modelo
l=[]
x=np.array([
[ 1.2032 , 0.34239826, -0.3272417 ],
[ 1.4500312 , 0.54530275, -0.64213556]
]) #weights
y=np.array([-1.7683319, 1.8029612, -1.7188473]) #array of biases
l.append(x)
l.append(y)
model.set_weights(l)

In [22]:
# observacion [0, 2]
observ = np.array([[0, 2]])
print(observ)

[[0 2]]


In [23]:
probs = model.predict_proba(observ)
probs

array([[0.14621755, 0.85144234, 0.00234022]], dtype=float32)

In [26]:
coefs_, intercept_ = model.get_weights()
print (coefs_)
print (intercept_)

[[ 1.2032      0.34239826 -0.3272417 ]
 [ 1.4500312   0.54530275 -0.64213556]]
[-1.7683319  1.8029612 -1.7188473]


In [27]:
h_s = coefs_.T.dot(observ.reshape(-1,1)) + intercept_.reshape(-1,1)
h_s

array([[ 1.13173044],
       [ 2.89356673],
       [-3.0031184 ]])

# RC2 - Pregunta 11

Calcular cada una de las probabilidades del problema anterior para la observación [0, 2]

In [29]:
p_ = np.exp(h_s) / np.exp(h_s).sum()
p_

array([[0.14621752],
       [0.85144225],
       [0.00234022]])

In [31]:
probs = model.predict_proba(observ)
probs

array([[0.14621755, 0.85144234, 0.00234022]], dtype=float32)

# RC2 - Pregunta 12

Dado el problema anterior calcular la loss (Entropía cruzada categórica) de la observación [0, 2] dado que se clasificó bien

In [35]:
# la mayor probabilidad es de la segunda clase, entonces valido contra el vector [0, 1, 0]
model.evaluate(observ, np.array([[0, 1, 0]]))

1/1 [==============================] - 0s 6ms/sample - loss: 0.1608 - accuracy: 1.0000


[0.1608235388994217, 1.0]

Cuanto valdría la loss si en realidad esa observación debería haberse clasificado como de la primer clase [1, 0, 0]

In [32]:
model.evaluate(observ, np.array([[1, 0, 0]]))

1/1 [==============================] - 0s 93ms/sample - loss: 1.9227 - accuracy: 0.0000e+00


[1.9226597547531128, 0.0]

# Pregunta 13

Dado el siguiente modelo multilabel, hallar las probabilidades de cada clase para la observación [0.5, 0.5]

In [52]:
# Instancio un modelo multilabel con función de activación sigmoidea y loss binary_xent

input_features=2
output_size = 4
model = Sequential()
model.add(Dense(output_size, name="Salida", activation='sigmoid', input_dim=input_features))
model.summary()

lr = 0.05 
# Lo compilo, notar que en vez de binary_crossentropy va categorical_crossentropy
model.compile(loss = 'binary_crossentropy', optimizer=Adam(lr=lr), metrics=['accuracy'])

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Salida (Dense)               (None, 4)                 12        
Total params: 12
Trainable params: 12
Non-trainable params: 0
_________________________________________________________________


In [53]:
coefs_, intercept_ = model.get_weights()
print(coefs_, intercept_)

[[ 0.9950235   0.89830256  0.94255066 -0.5261898 ]
 [ 0.49229598  0.4148593   0.4485836  -0.02210307]] [0. 0. 0. 0.]


In [54]:
new_coefs = np.array([
[-10.021508 , 9.780544 , -0.3323118 , 0.35372728],
[ 0.13239847, -0.1332541 , 8.324003 , -8.686057 ]
])
new_intercept = np.array([ 0.19760127, -0.18874079, 0.03835499, -0.04586953])

l = []
l.append(new_coefs)
l.append(new_intercept)

model.set_weights(l)

In [55]:
coefs_, intercept_ = model.get_weights()
print(coefs_, intercept_)

[[-10.021508     9.780544    -0.3323118    0.35372728]
 [  0.13239847  -0.1332541    8.324003    -8.686057  ]] [ 0.19760127 -0.18874079  0.03835499 -0.04586953]


In [58]:
observ = np.array([[0.5, 0.5]])
print(observ)

[[0.5 0.5]]


In [59]:
probs = model.predict_proba(observ)
probs

array([[0.00860343, 0.99038625, 0.982608  , 0.01459988]], dtype=float32)

# Pregunta 14

Calcular la loss para el problema anterior promediando las entropías cruzadas de cada probabilidad (suponer que se clasifico bien)

In [60]:
# si clasificó bien, asumo que el vector resultante es [0, 1, 1, 0] para calcular la loss

model.evaluate(observ, np.array([[0, 1, 1, 0]]))

1/1 [==============================] - 0s 105ms/sample - loss: 0.0126 - accuracy: 1.0000


[0.012638348154723644, 1.0]